In [1]:
import numpy as np

## jupyter notebook showing how to inject events with genie
This is fairly slow, and can be cpu intensive, so don't simulate more than 1,000 events like this. For large datasets use slurm with the py script

#### also the following cell was just to resolve some annoying import problems I got with vs code's conda environment

In [2]:
import os
import ctypes
import sys


modules_to_clean = []
for name in list(sys.modules.keys()):
    if any(substring in name for substring in ['h5py', 'prometheus', '_hdf5', '_h5py', '_errors']):
        modules_to_clean.append(name)

for mod in modules_to_clean:
    if mod in sys.modules:
        del sys.modules[mod]

# library path
intel_paths = [
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/mkl/lib/intel64",
]

current_ld_path = os.environ.get('LD_LIBRARY_PATH', '')
new_ld_path = ':'.join(intel_paths + [current_ld_path])
os.environ['LD_LIBRARY_PATH'] = new_ld_path

intel_libs = [
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libimf.so",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libintlc.so.5",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libsvml.so",
]

for lib_path in intel_libs:
    try:
        ctypes.CDLL(lib_path, mode=ctypes.RTLD_GLOBAL)
        print(f"Successfully preloaded: {lib_path}")
    except Exception as e:
        print(f"Failed to preload {lib_path}: {e}")

# Python paths
sys.path.insert(0, '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus')
sys.path.insert(0, '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples')

try:
    import h5py
    print("✓ h5py imported successfully")
except Exception as e:
    print(f"✗ h5py import failed: {e}")

try:
    from prometheus import Prometheus, config
    print("✓ prometheus imported successfully!")
except Exception as e:
    print(f"✗ prometheus import failed: {e}")
    import traceback
    traceback.print_exc()

Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libimf.so
Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libintlc.so.5
Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libsvml.so
✓ h5py imported successfully
✓ prometheus imported successfully!


In [3]:
from prometheus import Prometheus, config

In [4]:



import pandas as pd
import numpy as np
import time
import sys
import argparse
import os
import logging
from datetime import datetime

sys.path.append('../')
from prometheus import Prometheus, config
import prometheus
import gc

from genie_parser_injection import parse_and_convert_genie

from inject_in_cylinder import inject_particles_in_cylinder
from rotate_particles import rotate_particles_final
from gevgen_injection import run_gevgen, convert_to_gst, convert_to_gtrac

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
RESOURCE_DIR = f"{'/'.join(prometheus.__path__[0].split('/')[:-1])}/resources/"
OUTPUT_DIR = f"{'/'.join(prometheus.__path__[0].split('/')[:-1])}/genie_examples/output"


#### the gevgen injection happens in file, gevgen_injection.py and is called by helper functions in cell below 

In [5]:

def main(num_events, simset=1):
    # timing
    start_time = time.time()


    print(f"Using simset: {simset}")
    processing_start_time = time.time()



    logger.info("Starting GENIE simulation")
    genie_start = time.time()
    ghep_file = run_gevgen(num_events, OUTPUT_DIR)### TODO: add extra parameters to this in addition to just num_events
    gtrac_file = convert_to_gtrac(ghep_file, OUTPUT_DIR)
    
    genie_time = time.time() - genie_start
    logger.info(f"GENIE simulation completed in {genie_time:.2f} seconds")
    prometheus_set, primary_set = parse_and_convert_genie(gtrac_file)
    processing_end_time = time.time()
    print(f"File processing and conversion completed in {processing_end_time - processing_start_time:.2f} seconds")

    primrary_file_path = f'{OUTPUT_DIR}/genie_events_primary_new_test.parquet' ## could make these unique if you don't want them to be overwritten
    prometheus_file_path = f'{OUTPUT_DIR}/genie_events_prometheus_new_test.parquet'

    # inject into cylinder:
    primary_set, prometheus_set = inject_particles_in_cylinder(primary_set, ## neutrino information
                        prometheus_set, ## child particle information
                        cylinder_radius=500, ## meters
                        cylinder_height = 1000, # meters
                        cylinder_center = (0, 0, 0), # meters not including offset
                        )

    primary_set, prometheus_set = rotate_particles_final(primary_set, prometheus_set)


    save_start_time = time.time()
    # position arrays to serialized strings
    prometheus_set['position'] = prometheus_set['position'].apply(lambda x: [arr.tolist() for arr in x])
    num_events = len(prometheus_set)
    print(f"Processing {num_events} events")
    prometheus_set.to_parquet(prometheus_file_path)
    primary_set.to_parquet(primrary_file_path)
    save_end_time = time.time()
    print(f"Serialization and saving completed in {save_end_time - save_start_time:.2f} seconds")

    ## GENIE stuff:
    config["injection"]["name"] = "GENIE"
    config["run"]["outfile"] = f"{OUTPUT_DIR}/{num_events}_events_simset_{simset}.parquet"
    config["run"]["nevents"] = num_events
    config["injection"]["GENIE"]["inject"] = False ## ideally this should toggle using gevgen and be true, but for now setting this to false helps solve a headache in promtheus.py
    config["injection"]["GENIE"]["simulation"] = {}
    ## geofile:
    config["detector"]["geo file"] = f"{RESOURCE_DIR}/geofiles/icecube.geo"
    ## ppc configuration - using the 'new' ppc:
    config['photon propagator']['name'] = 'PPC_UPGRADE'
    config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpdir"] = "./ppc_tmpdir"+str(simset)
    config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpfile"] = "ppc_tmp"+str(simset)
    print(config["photon propagator"])
    config["photon propagator"]["PPC_UPGRADE"]["simulation"]["supress_output"] = False ## just for printing, can turn to True if you don't want a cluttered print out!

    # Timing for Prometheus simulation
    sim_start_time = time.time()
    p = Prometheus(config, primary_set_parquet_path=primrary_file_path, prometheus_set_parquet_path=prometheus_file_path)
    p.sim()
    sim_end_time = time.time()
    print(f"Prometheus simulation completed in {sim_end_time - sim_start_time:.2f} seconds")
    
    # End timing
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Total execution time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
    print('Finished without catastrophic error')
    return




In [6]:
num_events = 10 ## just a short dataset for testing
simset = 16
print("--------------------------------------------------------------")
print("--------------------------------------------------------------")
print("Launching simulation")
main(num_events, simset)
print("Finished call")
print("--------------------------------------------------------------")
print("--------------------------------------------------------------")

2025-05-07 14:32:50,820 - __main__ - INFO - Starting GENIE simulation
2025-05-07 14:32:50,821 - gevgen_injection - INFO - Running GEVGEN with 10 events
2025-05-07 14:32:50,821 - gevgen_injection - INFO - genie command: ['/groups/icecube/jackp/genie-3.4.2/bin/gevgen', '-n', '10', '-p', '14', '-t', '1000080160[0.888],1000010010[0.112]', '-e', '1,100', '-f', 'x^-2', '--seed', '12345', '--cross-sections', '/groups/icecube/jackp/genie-3.4.2/ice_numu_cross_sections.xml', '--event-generator-list', 'Default', '--event-record-print-level', '3', '-o', '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/gntp_icecube_10_events.ghep.root', '--debug-flags', 'PhysModels,EventGen,NucleonDecay', '--tune', 'G18_02a_00_000']


--------------------------------------------------------------
--------------------------------------------------------------
Launching simulation
Using simset: 16
cmd:  ['/groups/icecube/jackp/genie-3.4.2/bin/gevgen', '-n', '10', '-p', '14', '-t', '1000080160[0.888],1000010010[0.112]', '-e', '1,100', '-f', 'x^-2', '--seed', '12345', '--cross-sections', '/groups/icecube/jackp/genie-3.4.2/ice_numu_cross_sections.xml', '--event-generator-list', 'Default', '--event-record-print-level', '3', '-o', '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/gntp_icecube_10_events.ghep.root', '--debug-flags', 'PhysModels,EventGen,NucleonDecay', '--tune', 'G18_02a_00_000']


2025-05-07 14:33:20,140 - gevgen_injection - INFO - GEVGEN completed. Output: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/gntp_icecube_10_events.ghep.root
2025-05-07 14:33:20,141 - gevgen_injection - INFO - Converting to GTRAC format using rootracker




*****************************************************************************************************************
*                                                                                                               *
*                                                   .oooooo.    oooooooooooo ooooo      ooo ooooo oooooooooooo  *
*                                                  d8P'  `Y8b   `888'     `8 `888b.     `8' `888' `888'     `8  *
*                                                 888            888          8 `88b.    8   888   888          * 
*                        Ndyooym          dN      888            888oooo8     8   `88b.  8   888   888oooo8     *
*                     Nds//+sdmoy       d+m       888     ooooo  888    "     8     `88b.8   888   888    "     * 
*                   Nh+//ohN  m+s      N//syyN    `88.    .88'   888       o  8       `888   888   888       o  *
*                 Ny+//od   Nh+oN       o///+      `Y8bood8P'   o888ooooood8 o8o    

2025-05-07 14:33:21,371 - gevgen_injection - INFO - GTRAC conversion completed. Output: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-05-07 14:33:21,372 - __main__ - INFO - GENIE simulation completed in 30.55 seconds
2025-05-07 14:33:21,377 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-05-07 14:33:21,379 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-05-07 14:33:21,386 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-05-07 14:33:21,399 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-05-07 14:33:21,417 

...

returning hadron 2212
returning hadron 2212


Configuring ppc in "./ppc_tmpdir16/"
Configuring nextgen sensors in "./ppc_tmpdir16/"
Configuring icemodel in "./ppc_tmpdir16/"
Configuring tiltmodel in "./ppc_tmpdir16/"
Configuring holeice from "./ppc_tmpdir16/as.dat"
Configured: xR=1 eff=1.015 sf=0.137 g=0.9
Ice anisotropy is k(130)=1,1,1
New Ice anisotropy is 1.82212,0.740818,0.740818
Renorm. NI anisotropy 1.69545,0.689318,0.689318
Ice absorption anisotropy scaling is 0
Initialized BFR diffusion patterns; s_eff=0.00246854 m^-1
Loaded 12 angsens coefficients
Loaded 131072 random multipliers
Loaded 5083 cable positions
Loaded 399 RDE coefficients
 Found 4761 OMs of type -1,0
 Found 399 OMs of type -1,1
Loaded 5160 DOMs (21x19)
Loaded 80x125 2d dust layer points
Ice layer thickness: UNIFORM
Loaded 42 wavelenth points
Loaded 42 RDE coefficients
Calculated value of ppm=2268.22
Loaded 171 ice layers
Air bubble parameters: 11.7 1350 1400

Processing f2k muons from stdin on device 0
HQUO(photons/max number of hits)=1
photons: 2388  hits: 0

returning hadron 2212
Handling charged pion/kaon 211


Configuring ppc in "./ppc_tmpdir16/"
Configuring nextgen sensors in "./ppc_tmpdir16/"
Configuring icemodel in "./ppc_tmpdir16/"
Configuring tiltmodel in "./ppc_tmpdir16/"
Configuring holeice from "./ppc_tmpdir16/as.dat"
Configured: xR=1 eff=1.015 sf=0.137 g=0.9
Ice anisotropy is k(130)=1,1,1
New Ice anisotropy is 1.82212,0.740818,0.740818
Renorm. NI anisotropy 1.69545,0.689318,0.689318
Ice absorption anisotropy scaling is 0
Initialized BFR diffusion patterns; s_eff=0.00246854 m^-1
Loaded 12 angsens coefficients
Loaded 131072 random multipliers
Loaded 5083 cable positions
Loaded 399 RDE coefficients
 Found 4761 OMs of type -1,0
 Found 399 OMs of type -1,1
Loaded 5160 DOMs (21x19)
Loaded 80x125 2d dust layer points
Ice layer thickness: UNIFORM
Loaded 42 wavelenth points
Loaded 42 RDE coefficients
Calculated value of ppm=2268.22
Loaded 171 ice layers
Air bubble parameters: 11.7 1350 1400

Processing f2k muons from stdin on device 0
HQUO(photons/max number of hits)=1
photons: 753  hits: 0


Handling charged pion/kaon -211
Prometheus simulation completed in 18.22 seconds
Total execution time: 49.09 seconds (0.82 minutes)
Finished without catastrophic error
I am melting.... AHHHHHH!!!!
Finished call
--------------------------------------------------------------
--------------------------------------------------------------


In [7]:

primary_set_path = f'{OUTPUT_DIR}/genie_events_primary_new_test.parquet'
prometheus_set_path = f'{OUTPUT_DIR}/genie_events_prometheus_new_test.parquet'
out_parquet =  f"{OUTPUT_DIR}/{num_events}_events_simset_{simset}.parquet"

prometheus_set = pd.read_parquet(prometheus_set_path)
primary_set = pd.read_parquet(primary_set_path)
out_parquet = pd.read_parquet(out_parquet)

### the position on the primary set and prometheus set don't have the ~1950m z offset btw

In [8]:
primary_set

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,True,4.120269,14,NC,0.859495,1.626560,-1,-1,250.315228,-327.278036,-340.327872,"[250.31522761694274, -327.27803603986786, -340...",-1,"nu:14;tgt:1000080160;N:2212;proc:Weak[NC],QES;"
1,True,33.232259,14,NC,1.645141,3.146733,-1,-1,-246.193242,-264.566435,455.793266,"[-246.19324210669694, -264.5664346607915, 455....",-1,nu:14;tgt:1000010010;N:2212;q:2(v);proc:Weak[N...
2,True,26.600404,14,CC,0.217695,3.799162,-1,-1,136.090991,-290.720124,-231.006824,"[136.09099137591144, -290.72012375944047, -231...",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[C...
3,True,31.335799,14,CC,1.863040,2.431800,-1,-1,234.341716,277.273702,-341.054672,"[234.34171573039941, 277.2737024265705, -341.0...",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[C...
4,True,1.904728,14,CC,1.473233,6.070946,-1,-1,-269.798749,-401.475338,-422.953899,"[-269.79874892153396, -401.47533834656474, -42...",-1,"nu:14;tgt:1000080160;N:2212;proc:Weak[CC],RES;..."
5,True,2.360738,14,CC,2.449235,2.204709,-1,-1,-117.856536,-355.861112,-20.581127,"[-117.85653592138198, -355.86111182476117, -20...",-1,"nu:14;tgt:1000080160;N:2112;proc:Weak[CC],RES;..."
6,True,3.968209,14,CC,1.455142,4.815656,-1,-1,-136.285394,-113.588466,-397.939848,"[-136.28539415678978, -113.58846601516667, -39...",-1,"nu:14;tgt:1000080160;N:2112;proc:Weak[CC],QES;"
7,True,36.513016,14,NC,2.601142,4.734835,-1,-1,-256.092944,410.958391,410.973953,"[-256.0929444052748, 410.958391198653, 410.973...",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[N...
8,True,19.521958,14,CC,1.572520,4.732557,-1,-1,422.831582,7.568093,-287.359302,"[422.83158213542885, 7.568093061360401, -287.3...",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[C...
9,True,2.421808,14,CC,1.584321,1.459638,-1,-1,357.104208,0.408178,-378.930207,"[357.1042083152849, 0.40817835941985126, -378....",-1,nu:14;tgt:1000080160;N:2212;q:1(v);proc:Weak[C...


In [9]:
prometheus_set

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,"[False, False, False, False]","[0.00444, 3.9605257185188063, 1.05783024823171...","[22, 14, 2212, 2000000101]","[NC, NC, NC, NC]","[1.0789133318206938, 0.8643048877191286, 1.451...","[1.0468220344378236, 1.498706413321599, 2.6616...","[-1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0]","[250.31522761694274, 250.31522761694274, 250.3...","[-327.27803603986786, -327.27803603986786, -32...","[-340.3278718695565, -340.3278718695565, -340....","[[250.31522761694274, -327.27803603986786, -34...","[-1.0, -1.0, -1.0, -1.0]","[child, child, child, child]"
1,"[False, False, False, False, False, False]","[25.744678284252732, 1.5912865794041986, 5.187...","[14, 111, 2112, 111, 211, 111]","[NC, NC, NC, NC, NC, NC]","[1.6297597731793514, 1.6606910367118437, 1.702...","[3.118476779574891, 3.096243241764689, 3.04138...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-246.19324210669694, -246.19324210669694, -24...","[-264.5664346607915, -264.5664346607915, -264....","[455.7932662658518, 455.7932662658518, 455.793...","[[-246.19324210669694, -264.5664346607915, 455...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child]"
2,"[False, False, False, False, False, False, Fal...","[14.705120183999083, 0.5559631923089461, 2.614...","[13, 111, 2112, 211, 111, 111, 2212, -211]","[CC, CC, CC, CC, CC, CC, CC, CC]","[0.21597701230504598, 0.10836557324285931, 0.2...","[-2.4952662754188526, 3.0068660453823584, -2.5...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[136.09099137591144, 136.09099137591144, 136.0...","[-290.72012375944047, -290.72012375944047, -29...","[-231.00682403996495, -231.00682403996495, -23...","[[136.09099137591144, -290.72012375944047, -23...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, chi..."
3,"[False, False, False, False, False, False, False]","[19.904673998313992, 0.6045963532376314, 3.685...","[13, 211, 211, -211, 111, 2112, 111]","[CC, CC, CC, CC, CC, CC, CC]","[1.818746149387499, 2.046714726257454, 2.01102...","[2.3187309030757284, 2.600756492314741, 2.4435...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[234.34171573039941, 234.34171573039941, 234.3...","[277.2737024265705, 277.2737024265705, 277.273...","[-341.0546721856442, -341.0546721856442, -341....","[[234.34171573039941, 277.2737024265705, -341....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, child]"
4,"[False, False, False, False]","[0.00385, 0.4985908788035815, 1.86633738306353...","[22, 13, 2212, 211]","[CC, CC, CC, CC]","[1.9672439159060955, 2.8786792644566264, 1.185...","[0.4446294288605853, 1.1352377379170269, -0.26...","[-1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0]","[-269.79874892153396, -269.79874892153396, -26...","[-401.47533834656474, -401.47533834656474, -40...","[-422.95389873352474, -422.95389873352474, -42...","[[-269.79874892153396, -401.47533834656474, -4...","[-1.0, -1.0, -1.0, -1.0]","[child, child, child, child]"
5,"[False, False, False, False, False, False, False]","[0.00618, 1.0975641954166295, 1.12105649722162...","[22, 13, 2112, 2212, 2212, 211, -211]","[CC, CC, CC, CC, CC, CC, CC]","[1.948493925981121, 2.578138204272997, 2.00505...","[-0.6365089844385976, 1.9009147243179272, 0.95...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-117.85653592138198, -117.85653592138198, -11...","[-355.86111182476117, -355.86111182476117, -35...","[-20.581127460082655, -20.581127460082655, -20...","[[-117.85653592138198, -355.86111182476117, -2...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, child]"
6,"[False, False, False, False, False, False, Fal...","[2.9087030454378104, 0.96783604951397

### and the normal mc_truth and photons fields from the parquet:


In [10]:
out_parquet

,mc_truth,photons
0,"{'interaction': 2, 'initial_state_energy': 4.1...","{'sensor_pos_x': [], 'sensor_pos_y': [], 'sens..."
1,"{'interaction': 2, 'initial_state_energy': 33....","{'sensor_pos_x': [], 'sensor_pos_y': [], 'sens..."
2,"{'interaction': 1, 'initial_state_energy': 26....","{'sensor_pos_x': [106.94, 79.41], 'sensor_pos_..."
3,"{'interaction': 1, 'initial_state_energy': 31....","{'sensor_pos_x': [303.41, 174.47, 303.41], 'se..."
4,"{'interaction': 1, 'initial_state_energy': 1.9...","{'sensor_pos_x': [], 'sensor_pos_y': [], 'sens..."
5,"{'interaction': 1, 'initial_state_energy': 2.3...","{'sensor_pos_x': [], 'sensor_pos_y': [], 'sens..."
6,"{'interaction': 1, 'initial_state_energy': 3.9...","{'sensor_pos_x': [-9.68, -200.55], 'sensor_pos..."
7,"{'interaction': 2, 'initial_state_energy': 36....","{'sensor_pos_x': [], 'sensor_pos_y': [], 'sens..."
8,"{'interaction': 1, 'initial_state_energy': 19....","{'sensor_pos_x': [292.9, 292.9], 'sensor_pos_y..."
9,"{'interaction': 1, 'initial_state_energy': 2.4...","{'sensor_pos_x': [], 'sensor_pos_y': [], 'sens..."


In [12]:
out_parquet['mc_truth'][2]

{'interaction': 1,
 'initial_state_energy': 26.600404392730383,
 'initial_state_type': 14,
 'initial_state_zenith': 0.21769523800111712,
 'initial_state_azimuth': -2.4840235719107233,
 'initial_state_x': 141.96182083327568,
 'initial_state_y': -293.2387322865753,
 'initial_state_z': -2202.9825895438476,
 'final_state_energy': array([1.47051202e+01, 1.42527101e-02, 4.12656699e-02, 5.01399945e-02,
        5.55963192e-01, 2.61423463e+00, 7.62118454e-01, 7.30616473e+00,
        3.44948570e-01, 9.55530679e-01, 1.19694698e+00]),
 'final_state_type': array([  13.,   11.,   14.,  -12.,  111., 2112.,  211.,  111.,  111.,
        2212., -211.]),
 'final_state_zenith': array([0.21597701, 1.38551193, 2.13226019, 1.17465952, 0.10836557,
        0.21261091, 0.48523608, 0.20252897, 1.596404  , 0.18861782,
        0.26444112]),
 'final_state_azimuth': array([-2.49526628,  1.47899129,  2.21695426, -1.12969056,  3.00686605,
        -2.52034842, -2.98362076, -2.53897567,  0.02724833, -2.80848611,
       

In [13]:
out_parquet['photons'][2]

{'sensor_pos_x': array([106.94,  79.41]),
 'sensor_pos_y': array([  27.09, -248.24]),
 'sensor_pos_z': array([-2438.23, -2334.24]),
 'string_id': array([82, 18]),
 'sensor_id': array([58, 53]),
 't': array([3027.100586, 1023.673645]),
 'id_idx': array([1, 1])}

#### so I guess you don't need to load the prometheus and primary set if you have the mc_truth, but maybe something to keep in mind for one weight or anything needed to reweight to flux